In [1]:
import numpy as np
import pandas as pd

# Data Preperation

Load in the information from my database and remove goalkeepers because the statistics for goalkeepers are totally different. Instead of focusing on things like goals and assists, the statistics for goals are things like saves.

In [2]:
df = pd.read_csv('database.csv', index_col = 0)
df = df[df.position != 'Goalkeeper']

Familiarize ourselves with the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1521 entries, 0 to 1608
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1521 non-null   object 
 1   player_name           1521 non-null   object 
 2   age                   1521 non-null   int64  
 3   position              1521 non-null   object 
 4   club_involved_name    1521 non-null   object 
 5   fee                   1521 non-null   object 
 6   transfer_movement     1521 non-null   object 
 7   fee_cleaned           1521 non-null   float64
 8   league_name           1521 non-null   object 
 9   year                  1521 non-null   int64  
 10  season                1521 non-null   object 
 11  links                 1521 non-null   object 
 12  Season                1454 non-null   object 
 13  Matches               1454 non-null   float64
 14  Starts                1454 non-null   float64
 15  Minutes              

Many players don't have data on expected statistics. I filled expected goals with the actuals number of goals scored because in general they should be similar numbers. I did the same with assists and expected assists and so on.

In [4]:
df.xG.fillna(df.Goals, inplace = True)
df.npxG.fillna(df.Goals - df.Penalty_Goals, inplace = True)
df.xA.fillna(df.Assists, inplace = True)
df['xG/90'].fillna(df['Goals/90'], inplace = True)
df['xA/90'].fillna(df['Assists/90'], inplace = True)
df['xG+xA/90'].fillna(df['Goals+Assists/90'], inplace = True)
df['npxG/90'].fillna(df['Goals-PKs/90'], inplace = True)
df['npxG+xA/90'].fillna(df['Goals+Assists-PKs/90'], inplace = True)

I dropped the players with no season variable because these are the players who didn't have statistics listed on fbref for the year in question

In [5]:
df.dropna(subset = ['Season'], inplace = True)

I looked at df.info() again to try to find any null values that remained

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454 entries, 0 to 1608
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1454 non-null   object 
 1   player_name           1454 non-null   object 
 2   age                   1454 non-null   int64  
 3   position              1454 non-null   object 
 4   club_involved_name    1454 non-null   object 
 5   fee                   1454 non-null   object 
 6   transfer_movement     1454 non-null   object 
 7   fee_cleaned           1454 non-null   float64
 8   league_name           1454 non-null   object 
 9   year                  1454 non-null   int64  
 10  season                1454 non-null   object 
 11  links                 1454 non-null   object 
 12  Season                1454 non-null   object 
 13  Matches               1454 non-null   float64
 14  Starts                1454 non-null   float64
 15  Minutes              

I filled the remaining null values with zeros because I think that these null values were fbref's way of saying that no goals or assists were scored by that player that season

In [7]:
df.fillna(0, inplace = True)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454 entries, 0 to 1608
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1454 non-null   object 
 1   player_name           1454 non-null   object 
 2   age                   1454 non-null   int64  
 3   position              1454 non-null   object 
 4   club_involved_name    1454 non-null   object 
 5   fee                   1454 non-null   object 
 6   transfer_movement     1454 non-null   object 
 7   fee_cleaned           1454 non-null   float64
 8   league_name           1454 non-null   object 
 9   year                  1454 non-null   int64  
 10  season                1454 non-null   object 
 11  links                 1454 non-null   object 
 12  Season                1454 non-null   object 
 13  Matches               1454 non-null   float64
 14  Starts                1454 non-null   float64
 15  Minutes              

I removed commas from the minutes column as in 1,000 becomes 1000 so I could change it to an integer

In [9]:
df.Minutes = df.Minutes.apply(lambda x: str(x).replace(',', ''))

In [10]:
df.Minutes = df.Minutes.astype('int64')

In [11]:
df.position.value_counts()

Centre-Back           281
Centre-Forward        265
Central Midfield      207
Left Winger           123
Right Winger          117
Right-Back            116
Left-Back             114
Defensive Midfield    111
Attacking Midfield     73
Left Midfield          20
Second Striker         16
Right Midfield         11
Name: position, dtype: int64

I created three new columns is defence, is midfield and is attack, becuase in general attackers are more highly valued than defenders.

In [12]:
defence_dic = {'Centre-Back': 1, 'Centre-Forward': 0, 'Central Midfield': 0, 'Left Winger': 0, 'Right Winger': 0,
               'Right-Back': 1, 'Left-Back': 1, 'Defensive Midfield': 0, 'Goalkeeper': 1, 'Attacking Midfield': 0,
               'Left Midfield': 0, 'Second Striker': 0, 'Right Midfield': 0}
midfield_dic = {'Centre-Back': 0, 'Centre-Forward': 0, 'Central Midfield': 1, 'Left Winger': 0, 'Right Winger': 0,
                'Right-Back': 0, 'Left-Back': 0, 'Defensive Midfield': 1, 'Goalkeeper': 0, 'Attacking Midfield': 1,
                'Left Midfield': 1, 'Second Striker': 0, 'Right Midfield': 1}
attack_dic = {'Centre-Back': 0, 'Centre-Forward': 1, 'Central Midfield': 0, 'Left Winger': 1, 'Right Winger': 1,
              'Right-Back': 0, 'Left-Back': 0, 'Defensive Midfield': 0, 'Goalkeeper': 0, 'Attacking Midfield': 0,
              'Left Midfield': 0, 'Second Striker': 1, 'Right Midfield': 0}


In [13]:
df['is_defence'] = df.position.replace(defence_dic)
df['is_midfield'] = df.position.replace(midfield_dic)
df['is_attack'] = df.position.replace(attack_dic)

In [16]:
df_dummies = pd.get_dummies(df.position)
df = pd.concat([df, df_dummies], axis = 1)
df.drop(['position'], axis = 1, inplace = True)


In [17]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454 entries, 0 to 1608
Data columns (total 49 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   club_name             1454 non-null   object 
 1   player_name           1454 non-null   object 
 2   age                   1454 non-null   int64  
 3   club_involved_name    1454 non-null   object 
 4   fee                   1454 non-null   object 
 5   transfer_movement     1454 non-null   object 
 6   fee_cleaned           1454 non-null   float64
 7   league_name           1454 non-null   object 
 8   year                  1454 non-null   int64  
 9   season                1454 non-null   object 
 10  links                 1454 non-null   object 
 11  Season                1454 non-null   object 
 12  Matches               1454 non-null   float64
 13  Starts                1454 non-null   float64
 14  Minutes               1454 non-null   int64  
 15  Goals                

In [15]:
df

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,fee_cleaned,league_name,year,...,npxG,xA,xG/90,xA/90,xG+xA/90,npxG/90,npxG+xA/90,is_defence,is_midfield,is_attack
0,Feyenoord Rotterdam,Steven Berghuis,25,Right Winger,Watford,£5.85m,in,5.85,Eredivisie,2017,...,7.0,5.0,0.30,0.21,0.51,0.30,0.51,0,0,1
1,Feyenoord Rotterdam,Ridgeciano Haps,24,Left-Back,AZ Alkmaar,£5.40m,in,5.40,Eredivisie,2017,...,0.0,2.0,0.00,0.07,0.07,0.00,0.07,1,0,0
2,Feyenoord Rotterdam,Sofyan Amrabat,20,Central Midfield,FC Utrecht,£3.60m,in,3.60,Eredivisie,2017,...,0.0,5.0,0.00,0.18,0.18,0.00,0.18,0,1,0
3,ACF Fiorentina,Sofyan Amrabat,23,Central Midfield,Hellas Verona,£9.49m,in,9.49,Serie A,2019,...,1.0,0.0,0.08,0.00,0.08,0.08,0.08,0,1,0
4,Hellas Verona,Sofyan Amrabat,23,Central Midfield,Club Brugge,£3.15m,in,3.15,Serie A,2019,...,1.0,0.0,0.08,0.00,0.08,0.08,0.08,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1604,CA Osasuna,Marc Cardona,23,Centre-Forward,FC Barcelona,£2.25m,in,2.25,Primera Division,2019,...,2.7,0.6,0.41,0.10,0.50,0.41,0.50,0,0,1
1605,CA Osasuna,Robert Ibáñez,26,Right Winger,Getafe,£1.80m,in,1.80,Primera Division,2019,...,0.2,0.0,0.25,0.00,0.25,0.25,0.25,0,0,1
1606,Granada CF,Darwin Machís,26,Left Winger,Udinese Calcio,£2.70m,in,2.70,Primera Division,2019,...,1.3,0.6,0.26,0.12,0.38,0.26,0.38,0,0,1
1607,Granada CF,Domingos Duarte,24,Centre-Back,Sporting CP,£2.70m,in,2.70,Primera Division,2019,...,4.0,0.0,0.10,0.00,0.10,0.10,0.10,1,0,0


In [14]:
df.to_csv('data_cleaned.csv')